**INSTALLING THE REQ LIB AND PACKAGES**

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langchain_openai
!pip install langchain_pinecone
!pip install pinecone
!pip install openai
!pip install tiktoken
!pip install pypdf

  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 1.0.0
    Uninstalling langchain-text-splitters-1.0.0:
      Successfully uninstalled langchain-text-splitters-1.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.79 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
  Using cached langchain_core-1.0.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached langchain_text_splitters-1.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_core-1.0.0-py3-none-any.whl (467 kB)
Using cached langchain_text_splitters-1.0.0-py3-none-any.whl (33 kB)
  Attempting uninstall: langchain-core
    Found existing 

In [ ]:
import os  # No change; for environment variables [web:11]
from langchain_community.document_loaders import PyPDFDirectoryLoader  # Moved to community package for PDF directory loading
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Now in separate text_splitters module
from langchain_openai import OpenAIEmbeddings  # Embeddings moved to OpenAI integration package
from langchain_openai import ChatOpenAI  # Replaces deprecated langchain.llms.OpenAI use for chat completions
from langchain_pinecone import PineconeVectorStore  # Updated vectorstore for Pinecone integration
from langchain.chains import RetrievalQA  # Core chain remains in langchain
from langchain.prompts import PromptTemplate  # Prompt remains in core langchain
from langchain_core.documents import Document


/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


**EXTRACTING TEXT FROM PDF**

In [ ]:
loader=PyPDFDirectoryLoader("/content/PDF")
documents=loader.load()

**CONVERSION TO CHUNKS**

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks=text_splitter.split_documents(documents)

**INITIALIZING THE PINECONE**

In [ ]:
PINECONE_API_KEY = ''
OPENAI_API_KEY = ''
index_name = "project"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
embeddings = OpenAIEmbeddings()
docsearch = PineconeVectorStore.from_documents(
    text_chunks,
    embeddings,
    index_name=index_name
)


**QUERY USING SIMILARITY SEARCH**

In [ ]:
query= "aryan singh parihar studies in which college"

In [ ]:
docs = docsearch.similarity_search(query,k=3)

In [ ]:
docs

[Document(id='75a4a481-5bae-42a2-8f69-8ec8c703a5eb', metadata={'author': 'Aryan Parihar', 'creationdate': '2025-07-17T13:31:42+00:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2025-07-17T13:31:42+00:00', 'page': 0.0, 'page_label': '1', 'producer': 'www.ilovepdf.com', 'source': '/content/PDF/ARYAN SINGH PARIHAR.pdf', 'total_pages': 1.0}, page_content='ARYAN SINGH PARIHAR \nNEW DELHI,INDIA 110075 | aryanparihar0608@gmail.com | +91 9818141497 \n \nSUMMARY \n \nAs a final-year student pursuing B. Tech in Computer Science with specialization in Artificial Intelligence and Machine Learning from \nSRM Institute of Science and Technology. I’m skilled in Python, DSA, and SQL with practical knowledge of Generative AI, including \nLLMs, embedding’s, prompt engineering, and NLP. Experienced with tools like Open AI API, Gemini, Lang Chain, and Hugging Face.'),
 Document(id='5be13f0e-228d-40fc-b19d-9c75f80f9298', metadata={'author': 'Aryan Parihar', 'creationdate': '2025-07-17T13:31:42+00:00',

**LLM INTEGRATION**

In [ ]:
# this is done because the above output is not readeble
llm = ChatOpenAI()

In [ ]:
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=docsearch.as_retriever())

**ASKING THE QUERY**

In [ ]:
q1="aryan ompleted his schooling and college from where"
qa.run(q1)

/tmp/ipython-input-3848703678.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(q1)


'Based on the information provided, Aryan Singh Parihar completed his B. Tech in Computer Science with specialization in Artificial Intelligence and Machine Learning from SRM Institute of Science and Technology. However, there is no specific mention of where he completed his schooling.'

In [ ]:
q2="in which skills does aryan masters"
qa.run(q2)

'Aryan masters skills in Python, Data Structures and Algorithms (DSA), SQL, and has practical knowledge of Generative AI, including LLMs, embeddings, prompt engineering, and NLP. He is also experienced with tools like Open AI API, Gemini, Lang Chain, and Hugging Face.'

In [ ]:
while True:
    query = input("Ask a question (type 'exit' to quit): ")
    if query.lower() == 'exit':
        break
    response = qa.invoke(query)
    print(response)

Ask a question (type 'exit' to quit): who is aryan
{'query': 'who is aryan', 'result': 'Aryan Singh Parihar is a final-year student pursuing his B. Tech in Computer Science with a specialization in Artificial Intelligence and Machine Learning from SRM Institute of Science and Technology. He is skilled in Python, DSA, and SQL with practical knowledge of Generative AI, including LLMs, embeddings, prompt engineering, and NLP. Additionally, he has experience with tools like Open AI API, Gemini, Lang Chain, and Hugging Face.'}
Ask a question (type 'exit' to quit): exit
